# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-08-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-08-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-09 04:23:59,33.93911,67.709953,39616,1470,33058,5088.0,Afghanistan,101.766474,3.710622
1,NaN,NaN,NaN,Albania,2020-10-09 04:23:59,41.15330,20.168300,14899,411,9215,5273.0,Albania,517.721871,2.758574
2,NaN,NaN,NaN,Algeria,2020-10-09 04:23:59,28.03390,1.659600,52658,1783,36958,13917.0,Algeria,120.083803,3.386000
3,NaN,NaN,NaN,Andorra,2020-10-09 04:23:59,42.50630,1.521800,2568,54,1715,799.0,Andorra,3323.626480,2.102804
4,NaN,NaN,NaN,Angola,2020-10-09 04:23:59,-11.20270,17.873900,5958,208,2635,3115.0,Angola,18.128009,3.491104


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39254,39268,39285,39290,39297,39341,39422,39486,39548,39616
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,13518,13649,13806,13965,14117,14266,14410,14568,14730,14899
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,51368,51530,51690,51847,51995,52136,52270,52399,52520,52658


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1458,1458,1458,1458,1462,1462,1466,1467,1469,1470
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,384,387,388,389,392,396,400,403,407,411
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1726,1736,1741,1749,1756,1760,1768,1768,1771,1783


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32746,32789,32842,32842,32842,32852,32879,32977,33045,33058
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7732,7847,8077,8342,8536,8675,8825,8965,9115,9215
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,36063,36174,36282,36385,36482,36578,36672,36763,36857,36958


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5606,5640,5997,6024,6048,6073,6085,6116,6134,6141
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,886,896,898,902,921,921,921,923,927,927


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,27,27,28,27,27,27,27,27,27,28
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,50,52,53,53,53,53,53,55,56,64
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-10-09 04:23:59,32.539527,-86.644082,1863,28,0,1835.0,"Autauga, Alabama, US",3334.586264,1.502952
669,1075.0,Lamar,Alabama,US,2020-10-09 04:23:59,33.779950,-88.096680,404,3,0,401.0,"Lamar, Alabama, US",2926.475915,0.742574
670,1077.0,Lauderdale,Alabama,US,2020-10-09 04:23:59,34.901719,-87.656247,1880,37,0,1843.0,"Lauderdale, Alabama, US",2027.413215,1.968085


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7605873,212762,3021252
India,6835655,105526,5827704
Brazil,5028444,148957,4488092


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7605873,212762,3021252,4371859,2020-10-09 04:23:59,37.923092,-91.345036
India,6835655,105526,5827704,902425,2020-10-09 04:23:59,23.088275,81.806127
Brazil,5028444,148957,4488092,391395,2020-10-09 04:23:59,-12.669522,-48.480493
Russia,1253603,21939,998197,233467,2020-10-09 04:23:59,54.546312,62.120860
Colombia,886179,27180,773973,85026,2020-10-09 04:23:59,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,845270,16418,0
Texas,807754,16767,0
Florida,726013,15068,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,845270,16418,0,828852,2020-10-09 04:23:59,37.843962,-120.728594
Texas,807754,16767,0,790987,2020-10-09 04:23:59,31.660643,-98.653069
Florida,726013,15068,0,710945,2020-10-09 04:23:59,28.940755,-82.700744
New York,470104,33227,0,436877,2020-10-09 04:23:59,42.544151,-75.474183
Georgia,327407,7294,0,320113,2020-10-09 04:23:59,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,278665,6726,0
Florida,Miami-Dade,173721,3387,0
Illinois,Cook,151470,5279,0
Texas,Harris,149771,2657,0
Arizona,Maricopa,144867,3454,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,278665,6726,0,271939,2020-10-09 04:23:59,34.308284,-118.228241,6037.0
Florida,Miami-Dade,173721,3387,0,170334,2020-10-09 04:23:59,25.611236,-80.551706,12086.0
Illinois,Cook,151470,5279,0,146191,2020-10-09 04:23:59,41.841448,-87.816588,17031.0
Texas,Harris,149771,2657,0,147114,2020-10-09 04:23:59,29.858649,-95.393395,48201.0
Arizona,Maricopa,144867,3454,0,141413,2020-10-09 04:23:59,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,39486,14568,52399,2370,5725,107,824468,53083,27181,49819,...,532779,2177,59343,79796,1098,42432,10,2047,15170,7915
2020-10-07,39548,14730,52520,2568,5725,108,840915,53755,27206,50848,...,546952,2206,59579,80404,1099,42840,10,2049,15224,7919
2020-10-08,39616,14899,52658,2568,5958,111,856369,54473,27226,52057,...,564502,2226,60026,81019,1100,43256,10,2050,15301,7951


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,1467,403,1768,53,211,3,21827,990,897,822,...,42535,49,489,665,35,349,1,593,335,229
2020-10-07,1469,407,1771,53,211,3,22226,995,897,830,...,42605,49,491,671,35,355,1,593,335,229
2020-10-08,1470,411,1783,54,208,3,22710,1004,897,838,...,42682,49,496,678,35,359,1,593,335,229


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,32977,8965,36763,1615,2598,96,660272,44932,24915,39790,...,2417,1878,56058,70719,1023,35599,8,1327,14313,6440
2020-10-07,33045,9115,36857,1715,2598,97,670725,45110,24937,40499,...,2425,1890,56165,71531,1023,35953,8,1328,14342,6441
2020-10-08,33058,9215,36958,1715,2635,97,684844,45312,24969,41289,...,2474,1904,56837,72196,1023,36584,8,1329,14365,6446


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,382,382,386,393,397,400,402,403,402,404
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1751,1759,1772,1787,1807,1815,1832,1851,1862,1880
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,634,636,644,647,652,652,662,676,685,689
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5971,6017,6043,6068,6117,6141,6156,6173,6218,6243


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,160477,8752,0,221934,88071,838202,74169,59241,21466,49,...,205375,798569,79439,1821,1321,152945,90663,16942,136379,6770
2020-10-07,161418,8878,0,222538,88880,841928,74899,59364,21550,49,...,207455,803690,80446,1827,1322,153451,91208,17150,138698,6899
2020-10-08,161975,9005,0,223401,90145,845270,75761,59748,21682,49,...,209447,807754,81947,1838,1324,155270,91918,17333,141830,7092


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-06        1839    6116     923  691   1665     616    928    3730   
2020-10-07        1852    6134     927  703   1673     618    933    3743   
2020-10-08        1863    6141     927  708   1681     619    932    3752   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-06         1210      644  ...     272     51      346      127   
2020-10-07         1211      651  ...     284     54      352      127   
2020-10-08         1222      654  ...     288     54      369      129   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-06            371   629   368          0      120     46  
2020-10-07            378   633   368          0      119     50  
2020-10-08            382   645   373          0      123     51  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,2580,58,0,5713,1469,16244,2081,4521,646,0,...,2621,16528,488,58,20,3288,2165,370,1399,53
2020-10-07,2601,59,0,5733,1482,16338,2085,4522,649,0,...,2642,16661,496,58,20,3300,2177,375,1415,53
2020-10-08,2637,60,0,5743,1503,16418,2095,4527,651,0,...,2705,16767,501,58,20,3325,2183,376,1424,54


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-06          27      55       7   10     15      16     40      45   
2020-10-07          27      56       7   10     16      16     40      45   
2020-10-08          28      64       9   12     16      16     40      45   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-06           42       13  ...       2      1        4        1   
2020-10-07           43       13  ...       2      1        4        1   
2020-10-08           43       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-06              2     1     2          0        6      0  
2020-10-07              2     1     2          0        6      0  
2020-10-08              2     1     2          1        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.001623,0.010965,0.002468,0.000000,0.035262,0.000000,0.018204,0.007707,0.000294,0.018877,...,0.028090,0.010209,0.006735,0.008582,0.000912,0.011321,0.0,0.002940,0.005368,0.002152
2020-10-07,0.001570,0.011120,0.002309,0.083544,0.000000,0.009346,0.019949,0.012659,0.000920,0.020655,...,0.026602,0.013321,0.003977,0.007619,0.000911,0.009615,0.0,0.000977,0.003560,0.000505
2020-10-08,0.001719,0.011473,0.002628,0.000000,0.040699,0.027778,0.018378,0.013357,0.000735,0.023777,...,0.032087,0.009066,0.007503,0.007649,0.000910,0.009711,0.0,0.000488,0.005058,0.004041


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.000682,0.007500,0.000000,0.000000,0.060302,0.0,0.016723,0.006098,0.002235,0.004890,...,0.001790,0.020833,0.008247,0.010638,0.0,0.029499,0.0,0.001689,0.002994,0.004386
2020-10-07,0.001363,0.009926,0.001697,0.000000,0.000000,0.0,0.018280,0.005051,0.000000,0.009732,...,0.001646,0.000000,0.004090,0.009023,0.0,0.017192,0.0,0.000000,0.000000,0.000000
2020-10-08,0.000681,0.009828,0.006776,0.018868,-0.014218,0.0,0.021776,0.009045,0.000000,0.009639,...,0.001807,0.000000,0.010183,0.010432,0.0,0.011268,0.0,0.000000,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.002981,0.015864,0.002481,0.00000,0.002702,0.000000,0.017342,0.004965,0.001004,0.018741,...,0.003321,0.008593,0.007639,0.012702,0.000978,0.011853,0.0,0.003023,0.000489,0.002491
2020-10-07,0.002062,0.016732,0.002557,0.06192,0.000000,0.010417,0.015831,0.003962,0.000883,0.017819,...,0.003310,0.006390,0.001909,0.011482,0.000000,0.009944,0.0,0.000754,0.002026,0.000155
2020-10-08,0.000393,0.010971,0.002740,0.00000,0.014242,0.000000,0.021050,0.004478,0.001283,0.019507,...,0.020206,0.007407,0.011965,0.009297,0.000000,0.017551,0.0,0.000753,0.001604,0.000776


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,0.004784,0.016138,NaN,0.003908,0.007332,0.003226,0.008896,0.002047,0.004821,0.0,...,0.008228,0.005350,0.009095,0.002201,-0.000756,0.004090,0.004287,0.011523,0.015034,0.021270
2020-10-07,0.005864,0.014397,NaN,0.002722,0.009186,0.004445,0.009842,0.002076,0.003913,0.0,...,0.010128,0.006413,0.012676,0.003295,0.000757,0.003308,0.006011,0.012277,0.017004,0.019055
2020-10-08,0.003451,0.014305,NaN,0.003878,0.014233,0.003969,0.011509,0.006469,0.006125,0.0,...,0.009602,0.005057,0.018658,0.006021,0.001513,0.011854,0.007784,0.010671,0.022581,0.027975


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-06      0.004369  0.005094  0.002172  0.005822  0.001805  0.004894   
2020-10-07      0.007069  0.002943  0.004334  0.017366  0.004805  0.003247   
2020-10-08      0.005940  0.001141  0.000000  0.007112  0.004782  0.001618   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-06      0.001079  0.006748  0.001656  0.003115  ...  0.014925   
2020-10-07      0.005388  0.003485  0.000826  0.010870  ...  0.044118   
2020-10-08     -0.001072  0.002404  0.009083  0.004608  ...  0.014085   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-06      0.085106  0.032836  0.032520   0.022039  0.006400  0.008219   
2020-10-07      0.058824  0.017341  0.000000   0.018868  0.006359  0.000000   
2020-10-08      0.000000  0.048295  0.015748   0.010582  0.018957  0.013587   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-06            NaN -0.008264  0.045455  
2020-10-07            NaN -0.008333  0.086957  
2020-10-08            NaN  0.033613  0.020000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,0.008206,0.000000,NaN,0.001051,0.015204,0.004514,0.005800,0.000886,0.001550,NaN,...,0.009241,0.005536,0.012448,0.0,0.0,0.004583,0.003244,0.008174,0.013034,0.000000
2020-10-07,0.008140,0.017241,NaN,0.003501,0.008850,0.005787,0.001922,0.000221,0.004644,NaN,...,0.008012,0.008047,0.016393,0.0,0.0,0.003650,0.005543,0.013514,0.011437,0.000000
2020-10-08,0.013841,0.016949,NaN,0.001744,0.014170,0.004897,0.004796,0.001106,0.003082,NaN,...,0.023846,0.006362,0.010081,0.0,0.0,0.007576,0.002756,0.002667,0.006360,0.018868


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount   Bullock Butler   
2020-10-06      0.000000  0.037736  0.000000  0.0  0.000000  0.066667    0.0   
2020-10-07      0.000000  0.018182  0.000000  0.0  0.066667  0.000000    0.0   
2020-10-08      0.037037  0.142857  0.285714  0.2  0.000000  0.000000    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-10-06      0.022727  0.00000      0.0  ...     0.0    0.0      0.0   
2020-10-07      0.000000  0.02381      0.0  ...     0.0    0.0      0.0   
2020-10-08      0.000000  0.00000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-06          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-07          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-08          0.0        0.0   0.0   0.0        inf      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.001495,0.010694,0.002586,0.032208,0.027504,0.002170,0.015904,0.007349,0.000501,0.017665,...,0.028881,0.009142,0.006847,0.009090,0.000798,0.010847,6.847751e-34,0.001567,0.004709,0.001728
2020-10-07,0.001532,0.010907,0.002447,0.057876,0.013752,0.005758,0.017927,0.010004,0.000710,0.019160,...,0.027742,0.011232,0.005412,0.008355,0.000855,0.010231,3.423875e-34,0.001272,0.004134,0.001117
2020-10-08,0.001626,0.011190,0.002537,0.028938,0.027225,0.016768,0.018152,0.011680,0.000723,0.021468,...,0.029914,0.010149,0.006457,0.008002,0.000882,0.009971,1.711938e-34,0.000880,0.004596,0.002579


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.001208,0.008282,0.001931,2.668805e-19,0.039048,2.104411e-53,0.019599,0.006063,0.001665,0.005235,...,0.001269,0.010501,0.008422,0.009338,2.314935e-12,0.024463,0.0,0.001799,0.002295,0.002202
2020-10-07,0.001286,0.009104,0.001814,1.334403e-19,0.019524,1.052205e-53,0.018939,0.005557,0.000832,0.007484,...,0.001457,0.005251,0.006256,0.009180,1.157468e-12,0.020828,0.0,0.000899,0.001148,0.001101
2020-10-08,0.000983,0.009466,0.004295,9.433962e-03,0.002653,5.261027e-54,0.020358,0.007301,0.000416,0.008561,...,0.001632,0.002625,0.008220,0.009806,5.787338e-13,0.016048,0.0,0.000450,0.000574,0.000551


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-06,0.001782,0.017468,0.002561,0.015580,0.020015,0.002009,0.01787,0.003454,0.000793,0.016807,...,0.003045,0.008450,0.007778,0.012931,0.001348,0.016242,4.012354e-36,0.003180,0.003779,0.004803
2020-10-07,0.001922,0.017100,0.002559,0.038750,0.010007,0.006213,0.01685,0.003708,0.000838,0.017313,...,0.003177,0.007420,0.004843,0.012207,0.000674,0.013093,2.006177e-36,0.001967,0.002902,0.002479
2020-10-08,0.001158,0.014036,0.002650,0.019375,0.012125,0.003106,0.01895,0.004093,0.001061,0.018410,...,0.011692,0.007414,0.008404,0.010752,0.000337,0.015322,1.003089e-36,0.001360,0.002253,0.001628


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,0.004931,0.019198,NaN,0.002853,0.006834,0.003463,0.008060,0.004968,0.005520,1.324046e-62,...,0.008955,0.005235,0.012152,0.006717,-0.001167,0.004830,0.004761,0.010200,0.015264,0.021019
2020-10-07,0.005398,0.016797,NaN,0.002787,0.008010,0.003954,0.008951,0.003522,0.004717,6.620229e-63,...,0.009541,0.005824,0.012414,0.005006,-0.000205,0.004069,0.005386,0.011239,0.016134,0.020037
2020-10-08,0.004424,0.015551,NaN,0.003333,0.011121,0.003962,0.010230,0.004995,0.005421,3.310115e-63,...,0.009572,0.005440,0.015536,0.005513,0.000654,0.007962,0.006585,0.010955,0.019358,0.024006


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-06      0.003962  0.005044  0.002681  0.005446  0.002635  0.002651   
2020-10-07      0.005516  0.003994  0.003508  0.011406  0.003720  0.002949   
2020-10-08      0.005728  0.002567  0.001754  0.009259  0.004251  0.002283   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-06      0.001367  0.006835  0.002740  0.004057  ...  0.014511   
2020-10-07      0.003377  0.005160  0.001783  0.007463  ...  0.029314   
2020-10-08      0.001153  0.003782  0.005433  0.006036  ...  0.021699   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-06      0.074280  0.030171  0.019989   0.016492  0.011280  0.008326   
2020-10-07      0.066552  0.023756  0.009994   0.017680  0.008820  0.004163   
2020-10-08      0.033276  0.036026  0.012871   0.014131  0.013889  0.008875   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-06      -0.571428  0.001977  0.041514  
2020-10-07      -0.571428 -0.003178  0.064235  
2020-10-08      -0.571428  0.015218  0.042118  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-06,0.004570,0.001485,NaN,0.000931,0.014344,0.003458,0.003694,0.000697,0.001386,NaN,...,0.009167,0.005415,0.010208,3.834469e-21,0.000103,0.003417,0.003720,0.007426,0.009162,0.000939
2020-10-07,0.006355,0.009363,NaN,0.002216,0.011597,0.004622,0.002808,0.000459,0.003015,NaN,...,0.008589,0.006731,0.013301,1.917235e-21,0.000051,0.003533,0.004632,0.010470,0.010300,0.000469
2020-10-08,0.010098,0.013156,NaN,0.001980,0.012883,0.004760,0.003802,0.000782,0.003048,NaN,...,0.016218,0.006547,0.011691,9.586173e-22,0.000026,0.005554,0.003694,0.006568,0.008330,0.009669


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-10-06     -0.000224  0.019484  2.974477e-16 -0.000639  0.000005   
2020-10-07     -0.000112  0.018833  1.487238e-16 -0.000319  0.033336   
2020-10-08      0.018463  0.080845  1.428571e-01  0.099840  0.016668   

Province_State                                                        ...  \
Admin2           Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-10-06      0.032980  0.000102  0.011397  2.384487e-08 -0.000036  ...   
2020-10-07      0.016490  0.000051  0.005698  1.190477e-02 -0.000018  ...   
2020-10-08      0.008245  0.000025  0.002849  5.952387e-03 -0.000009  ...   

Province_State       Wyoming                                                 \
Admin2                  Park Platte      Sheridan Sublette Sweetwater Teton   
2020-10-06      9.536743e-07    0.0  1.430511e-06      0.0        0.0   0.0   
2020-10-07      4.768372e-07    0.0  7.152557e-07      0.0        0.0   0.0   
2020-10-08      2.384186e-07    0.0  3.576279e-07      0.0        0.0   0.0   

Province_State                                   
Admin2         Uinta Unassigned Washakie Weston  
2020-10-06       0.0  -0.999969      0.0    NaN  
2020-10-07       0.0  -0.999969      0.0    NaN  
2020-10-08       0.0  -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201008,AK,9996,NaN,481175,NaN,491171,46.0,NaN,NaN,...,491171,1,0,7232d5fe7f4dd05983c93f2954527e68736578c6,0,0,0,0,0,NaN
1,20201008,AL,161975,19311.0,1042046,NaN,1184710,754.0,17989.0,NaN,...,1204021,36,74,6ffc9058ab0b1b61e62ff3a69eb2964a1b867624,0,0,0,0,0,NaN
2,20201008,AR,90145,4165.0,1025546,NaN,1111526,546.0,5805.0,237.0,...,1115691,21,65,47eef11f062e3f441963448617de25ea53b06c00,0,0,0,0,0,NaN
3,20201008,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,...,1616,0,0,c06d3835887c262fa9fc5235310a0fb719c99889,0,0,0,0,0,NaN
4,20201008,AZ,223401,4805.0,1312499,NaN,1531095,728.0,20171.0,156.0,...,1535900,10,121,df15c456449f04fc6bd50c0a33df72e67f06aa30,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201008,AK,9996.0,NaN,481175.0,NaN,491171.0,46.0,NaN,NaN,...,491171,1,0,7232d5fe7f4dd05983c93f2954527e68736578c6,0,0,0,0,0,NaN
1,20201008,AL,161975.0,19311.0,1042046.0,NaN,1184710.0,754.0,17989.0,NaN,...,1204021,36,74,6ffc9058ab0b1b61e62ff3a69eb2964a1b867624,0,0,0,0,0,NaN
2,20201008,AR,90145.0,4165.0,1025546.0,NaN,1111526.0,546.0,5805.0,237.0,...,1115691,21,65,47eef11f062e3f441963448617de25ea53b06c00,0,0,0,0,0,NaN
3,20201008,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,...,1616,0,0,c06d3835887c262fa9fc5235310a0fb719c99889,0,0,0,0,0,NaN
4,20201008,AZ,223401.0,4805.0,1312499.0,NaN,1531095.0,728.0,20171.0,156.0,...,1535900,10,121,df15c456449f04fc6bd50c0a33df72e67f06aa30,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-08,AK,9996,NaN,481175,NaN,491171,46.0,NaN,NaN,NaN,...,491171,1,0,7232d5fe7f4dd05983c93f2954527e68736578c6,0,0,0,0,0,NaN
2020-10-08,AL,161975,19311.0,1042046,NaN,1184710,754.0,17989.0,NaN,1876.0,...,1204021,36,74,6ffc9058ab0b1b61e62ff3a69eb2964a1b867624,0,0,0,0,0,NaN
2020-10-08,AR,90145,4165.0,1025546,NaN,1111526,546.0,5805.0,237.0,NaN,...,1115691,21,65,47eef11f062e3f441963448617de25ea53b06c00,0,0,0,0,0,NaN
2020-10-08,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,NaN,...,1616,0,0,c06d3835887c262fa9fc5235310a0fb719c99889,0,0,0,0,0,NaN
2020-10-08,AZ,223401,4805.0,1312499,NaN,1531095,728.0,20171.0,156.0,NaN,...,1535900,10,121,df15c456449f04fc6bd50c0a33df72e67f06aa30,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-08,AK,9996.0,NaN,481175.0,NaN,491171.0,46.0,NaN,NaN,NaN,...,491171,1,0,7232d5fe7f4dd05983c93f2954527e68736578c6,0,0,0,0,0,NaN
2020-10-08,AL,161975.0,19311.0,1042046.0,NaN,1184710.0,754.0,17989.0,NaN,1876.0,...,1204021,36,74,6ffc9058ab0b1b61e62ff3a69eb2964a1b867624,0,0,0,0,0,NaN
2020-10-08,AR,90145.0,4165.0,1025546.0,NaN,1111526.0,546.0,5805.0,237.0,NaN,...,1115691,21,65,47eef11f062e3f441963448617de25ea53b06c00,0,0,0,0,0,NaN
2020-10-08,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,...,1616,0,0,c06d3835887c262fa9fc5235310a0fb719c99889,0,0,0,0,0,NaN
2020-10-08,AZ,223401.0,4805.0,1312499.0,NaN,1531095.0,728.0,20171.0,156.0,NaN,...,1535900,10,121,df15c456449f04fc6bd50c0a33df72e67f06aa30,0,0,0,0,0,NaN
